### import libraries

#### import via ivy

In [4]:
import $ivy.`org.scalanlp:breeze_2.11:0.13.2`
// import $ivy.`org.jfree:jfreechart:1.5.0`
import $ivy.`org.apache.spark:spark-core_2.11:2.4.3`
import $ivy.`org.apache.spark:spark-sql_2.11:2.4.3`
import $ivy.`org.apache.spark:spark-mllib_2.11:2.4.3`
import $ivy.`org.apache.spark:spark-graphx_2.11:2.4.3`
// import $ivy.`org.apache.spark:spark-streaming_2.11:2.4.3`
//import $ivy.`sh.almond::ammonite-spark:0.3.0`

import $ivy.$                                
// import $ivy.`org.jfree:jfreechart:1.5.0`

import $ivy.$                                       

import $ivy.$                                      

import $ivy.$                                        

import $ivy.$                                         
// import $ivy.`org.apache.spark:spark-streaming_2.11:2.4.3`
//import $ivy.`sh.almond::ammonite-spark:0.3.0`

#### Scala Library for Matrix Calculation

In [2]:
import breeze.linalg.csvread
import breeze.linalg.{DenseMatrix => DM}
import breeze.linalg.{DenseVector => DV}
import breeze.linalg

import breeze.linalg.csvread

import breeze.linalg.{DenseMatrix => DM}

import breeze.linalg.{DenseVector => DV}

import breeze.linalg

#### File IO

In [3]:
import java.io._
import scala.io.Source

import java.io._

import scala.io.Source

#### Spark Libraries

In [5]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkConf
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Column
import org.apache.spark.sql.catalyst.plans.{Inner, LeftOuter, Cross}

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.SparkSession

import org.apache.spark.SparkConf

import org.apache.spark.sql.functions._

import org.apache.spark.sql.Column

import org.apache.spark.sql.catalyst.plans.{Inner, LeftOuter, Cross}

#### Set level or warning statemens

In [6]:
import org.apache.log4j.Logger
import org.apache.log4j.Level
Logger.getLogger("org").setLevel(Level.ERROR)

import org.apache.log4j.Logger

import org.apache.log4j.Level


#### Specify folder & file paths 

In [7]:
val folder = new File("/Users/tomoya/Documents/dataset/champs-scalar-coupling")
val files = folder.listFiles

folder: File = /Users/tomoya/Documents/dataset/champs-scalar-coupling
files: Array[File] = Array(
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/.DS_Store,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/dipole_moments.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/magnetic_shielding_tensors.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/mulliken_charges.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/potential_energy.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/sample_submission.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/scalar_coupling_contributions.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/structures.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/structures.zip,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/test.csv,
  /Users/tomoya/Documents/dataset/champs-scalar-coupling/train.csv
)

In [13]:
val trainDataFile = files.filter(_.getName.matches(""".*train.csv""")).head

trainDataFile: File = /Users/tomoya/Documents/dataset/champs-scalar-coupling/train.csv

files.filter(x => x.getName.matches("""train.csv"""))

In [14]:
// Function to read csv file (UTF-8)
def readHead(path:File, nline:Int):Seq[String] = {
    val reader = new BufferedReader(new InputStreamReader(new FileInputStream(path), "UTF-8"))
    try {
        Iterator.continually(reader.readLine()).take(nline).toList // Not toSeq
    }finally{
        reader.close()
    }
}

defined function readHead

In [15]:
val list = readHead(trainDataFile, 10000)

list: Seq[String] = List(
  "id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant",
  "0,dsgdb9nsd_000001,1,0,1JHC,84.8076",
  "1,dsgdb9nsd_000001,1,2,2JHH,-11.257",
  "2,dsgdb9nsd_000001,1,3,2JHH,-11.2548",
  "3,dsgdb9nsd_000001,1,4,2JHH,-11.2543",
  "4,dsgdb9nsd_000001,2,0,1JHC,84.8074",
  "5,dsgdb9nsd_000001,2,3,2JHH,-11.2541",
  "6,dsgdb9nsd_000001,2,4,2JHH,-11.2548",
  "7,dsgdb9nsd_000001,3,0,1JHC,84.8093",
  "8,dsgdb9nsd_000001,3,4,2JHH,-11.2543",
  "9,dsgdb9nsd_000001,4,0,1JHC,84.8095",
  "10,dsgdb9nsd_000002,1,0,1JHN,32.6889",
  "11,dsgdb9nsd_000002,1,2,2JHH,-11.1866",
  "12,dsgdb9nsd_000002,1,3,2JHH,-11.1757",
  "13,dsgdb9nsd_000002,2,0,1JHN,32.6891",
  "14,dsgdb9nsd_000002,2,3,2JHH,-11.1758",
  "15,dsgdb9nsd_000002,3,0,1JHN,32.6905",
  "16,dsgdb9nsd_000003,1,2,2JHH,-9.94641",
  "17,dsgdb9nsd_000005,2,0,1JHC,171.22",
  "18,dsgdb9nsd_000005,2,1,2JHN,5.18246",
  "19,dsgdb9nsd_000007,2,0,1JHC,83.5429",
  "20,dsgdb9nsd_000007,2,1,2JHC,-2.37831",
  "21,dsgdb9nsd

In [16]:
val cols = list(0).split(",")

cols: Array[String] = Array(
  "id",
  "molecule_name",
  "atom_index_0",
  "atom_index_1",
  "type",
  "scalar_coupling_constant"
)

#### Init Spark

In [17]:
val sc = new SparkConf
sc.setAppName("chemical").setMaster("local[*]")
val spark = SparkSession.builder().config(sc).getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


sc: SparkConf = org.apache.spark.SparkConf@43b4bb6d
res16_1: SparkConf = org.apache.spark.SparkConf@43b4bb6d
spark: SparkSession = org.apache.spark.sql.SparkSession@619dbd13

In [18]:
def toRecord(line:String) = {
    val strs = line.split(",")
    (strs(0).toInt, strs(1), strs(2).toInt, strs(3).toInt, strs(4), strs(5).toDouble)
}

defined function toRecord

In [19]:
val sampleDF = spark.createDataFrame((1 to 999).map(x => list(x)).map(x => toRecord(x))).toDF(cols:_*)

sampleDF: DataFrame = [id: int, molecule_name: string ... 4 more fields]

#### Drop unneeded column information

In [20]:
val sampleDF_2 = sampleDF.select(col("*") +: ((1 to 4).map(x => col("type").substr(x, 1).as("t1_" + x.toString)) ++
                            (1 to 3).map(x => col("type").substr(x, 2).as("t2_" + x.toString))):_*)

sampleDF_2: DataFrame = [id: int, molecule_name: string ... 11 more fields]

In [21]:
sampleDF_2.show()

+---+----------------+------------+------------+----+------------------------+----+----+----+----+----+----+----+
| id|   molecule_name|atom_index_0|atom_index_1|type|scalar_coupling_constant|t1_1|t1_2|t1_3|t1_4|t2_1|t2_2|t2_3|
+---+----------------+------------+------------+----+------------------------+----+----+----+----+----+----+----+
|  0|dsgdb9nsd_000001|           1|           0|1JHC|                 84.8076|   1|   J|   H|   C|  1J|  JH|  HC|
|  1|dsgdb9nsd_000001|           1|           2|2JHH|                 -11.257|   2|   J|   H|   H|  2J|  JH|  HH|
|  2|dsgdb9nsd_000001|           1|           3|2JHH|                -11.2548|   2|   J|   H|   H|  2J|  JH|  HH|
|  3|dsgdb9nsd_000001|           1|           4|2JHH|                -11.2543|   2|   J|   H|   H|  2J|  JH|  HH|
|  4|dsgdb9nsd_000001|           2|           0|1JHC|                 84.8074|   1|   J|   H|   C|  1J|  JH|  HC|
|  5|dsgdb9nsd_000001|           2|           3|2JHH|                -11.2541|   2|   J|

In [22]:
val summaries = sampleDF_2.summary()

summaries: DataFrame = [summary: string, id: string ... 12 more fields]

In [23]:
summaries.show()

+-------+-----------------+----------------+-----------------+------------------+----+------------------------+------------------+----+----+----+----+----+----+
|summary|               id|   molecule_name|     atom_index_0|      atom_index_1|type|scalar_coupling_constant|              t1_1|t1_2|t1_3|t1_4|t2_1|t2_2|t2_3|
+-------+-----------------+----------------+-----------------+------------------+----+------------------------+------------------+----+----+----+----+----+----+
|  count|              999|             999|              999|               999| 999|                     999|               999| 999| 999| 999| 999| 999| 999|
|   mean|            499.0|            null|6.367367367367367|3.2412412412412412|null|       20.20008205105104|2.1731731731731734|null|null|null|null|null|null|
| stddev|288.5307609250702|            null|2.370953186843761|3.1443018508277945|null|       41.64950520655079|0.7537754279841617|null|null|null|null|null|null|
|    min|                0|dsgdb9n

### Regression test

#### Setting Params

In [167]:
import org.apache.spark.sql.types._

import org.apache.spark.sql.types._

In [41]:
import org.apache.spark.ml.{Pipeline, PipelineModel} 
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import spark.implicits._

var trainSample = 1.0
var testSample = 1.0
val seed = 12345L
val numFolds = 10

import org.apache.spark.ml.regression.{LinearRegression}

import org.apache.spark.ml.{Pipeline, PipelineModel} 

import org.apache.spark.ml.evaluation.RegressionEvaluator

import org.apache.spark.ml.tuning.ParamGridBuilder

import org.apache.spark.ml.tuning.CrossValidator

import org.apache.spark.mllib.evaluation.RegressionMetrics

import org.apache.spark.ml.feature.VectorAssembler


import org.apache.spark.ml.linalg.Vectors

import org.apache.spark.ml.regression.GeneralizedLinearRegression

import spark.implicits._


trainSample: Double = 1.0
testSample: Double = 1.0
seed: Long = 12345L
numFolds: Int = 10
MaxIter: Seq[Int] = List(1000)
RegParam: Seq[Double] = List(0.001)
Tol: Seq[Double] = List(1.0E-6)
ElasticNetParam: Seq[Double] = List(0.001)

### Read and define training / validation / test data

In [42]:
val targetCol = sampleDF.columns.filter(_.matches("""^scal.*""")).head

targetCol: String = "scalar_coupling_constant"

In [168]:
var trainAll = spark.read.option("header", "true").csv(trainDataFile.getAbsolutePath).
    withColumnRenamed(targetCol, "label").
    withColumn("label", col("label").cast(DoubleType)).sample(false, trainSample)

// var DF = trainAll.na.drop()
// if (trainAll == DF)
//     println("No null values in the DataFrame") 
// else{
//     println("Null values exist in the DataFrame")
//     trainAll = DF 
// }
val seed = 12345L
val splits = trainAll.randomSplit(Array(0.75, 0.25), seed)
val (trainingData, validationData) = (splits(0), splits(1))

trainingData.cache
validationData.cache

val testDataFile = files.filter(_.getName.matches("""test.csv""")).head
val testData = spark.read.option("header", "true").csv(testDataFile.getAbsolutePath).sample(false, testSample).cache

trainAll: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, molecule_name: string ... 4 more fields]
seed: Long = 12345L
splits: Array[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = Array(
  [id: string, molecule_name: string ... 4 more fields],
  [id: string, molecule_name: string ... 4 more fields]
)
trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, molecule_name: string ... 4 more fields]
validationData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, molecule_name: string ... 4 more fields]
res167_4: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, molecule_name: string ... 4 more fields]
res167_5: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, molecule_name: string ... 4 more fields]
testDataFile: File = /Users/tomoya/Documents/dataset/champs-scalar-coupling/test.csv
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, molecu

In [141]:
import scala.collection.mutable.ListBuffer
val preprocessingList = new ListBuffer[DataFrame => DataFrame]()

import scala.collection.mutable.ListBuffer


preprocessingList: ListBuffer[DataFrame => DataFrame] = ListBuffer()

+ Define function for renaming columns

In [142]:
def processTypeColumn(df:DataFrame):DataFrame = df.select(col("*")+: Seq(1,4).map( x => col("type").substr(x, 1).as("t" + x.toString)):_*).drop("type")
preprocessingList += processTypeColumn
trainingData.transform(processTypeColumn).show

+-------+----------------+------------+------------+---------+---+---+
|     id|   molecule_name|atom_index_0|atom_index_1|    label| t1| t4|
+-------+----------------+------------+------------+---------+---+---+
|      0|dsgdb9nsd_000001|           1|           0|  84.8076|  1|  C|
|      1|dsgdb9nsd_000001|           1|           2|  -11.257|  2|  H|
|     10|dsgdb9nsd_000002|           1|           0|  32.6889|  1|  N|
|    100|dsgdb9nsd_000012|           3|           5|  12.0536|  3|  H|
|  10000|dsgdb9nsd_000524|          14|           5|   86.787|  1|  C|
| 100000|dsgdb9nsd_003735|           7|           8| -11.7083|  2|  H|
|1000002|dsgdb9nsd_034940|          15|           5|-0.342991|  2|  C|
|1000003|dsgdb9nsd_034940|          15|           6| -1.57302|  2|  C|
|1000004|dsgdb9nsd_034940|          15|           7|   104.79|  1|  C|
|1000005|dsgdb9nsd_034940|          15|           8|  4.37176|  3|  C|
|1000006|dsgdb9nsd_034940|          16|           1| 0.538655|  3|  C|
|10000

defined function processTypeColumn
res141_1: ListBuffer[DataFrame => DataFrame] = ListBuffer(<function1>)

+ Define function for make 1 additional feature

In [143]:
def processAtomIndex(df:DataFrame):DataFrame = df.select(col("*"), abs(col("atom_index_0") - col("atom_index_1")).as("atom_index_diff"))
preprocessingList += processAtomIndex
trainingData.transform(processAtomIndex).show

+-------+----------------+------------+------------+----+---------+---------------+
|     id|   molecule_name|atom_index_0|atom_index_1|type|    label|atom_index_diff|
+-------+----------------+------------+------------+----+---------+---------------+
|      0|dsgdb9nsd_000001|           1|           0|1JHC|  84.8076|            1.0|
|      1|dsgdb9nsd_000001|           1|           2|2JHH|  -11.257|            1.0|
|     10|dsgdb9nsd_000002|           1|           0|1JHN|  32.6889|            1.0|
|    100|dsgdb9nsd_000012|           3|           5|3JHH|  12.0536|            2.0|
|  10000|dsgdb9nsd_000524|          14|           5|1JHC|   86.787|            9.0|
| 100000|dsgdb9nsd_003735|           7|           8|2JHH| -11.7083|            1.0|
|1000002|dsgdb9nsd_034940|          15|           5|2JHC|-0.342991|           10.0|
|1000003|dsgdb9nsd_034940|          15|           6|2JHC| -1.57302|            9.0|
|1000004|dsgdb9nsd_034940|          15|           7|1JHC|   104.79|         

defined function processAtomIndex
res142_1: ListBuffer[DataFrame => DataFrame] = ListBuffer(
  <function1>,
  <function1>
)

### Exploratory analysis (average target values by categorical variables)

In [70]:
var categoricalCols = trainingData.transform(processTypeColumn).transform(processAtomIndex).columns.filter(x => x.matches("""^(atom.*\d{1}|t\d{1})"""))
var tempAggs:Map[String, Column=>Column] = Map(
    "count" -> count,
    "min" -> max, 
    "max" -> min, 
    "mean"-> avg, 
    "stddev"-> stddev
)
var tempExpr = tempAggs.map(x => x._2(col("label"))).toSeq

In [71]:
import scala.util.matching.Regex.Match

def replacePars(str:String) = "[(|)]".r replaceSomeIn(str, { 
  case Match("(") => Some("_") 
  case Match(")") => Some("") 
  case _ => None})

def replaceColumnNameChars(df:DataFrame):DataFrame = {
   df.select(df.columns.map(x => col(x).as(replacePars(x))):_*)
}

val aggrList01 = categoricalCols.map(x => {
    trainingData.transform(processTypeColumn).transform(processAtomIndex).
    groupBy(x).
    agg(tempExpr.head, tempExpr.tail:_*).
    transform(replaceColumnNameChars).
    sort(col("avg_label").desc)
})

In [72]:
aggrList01(0).show

+------------+-----------+------------------+---------+---------+------------------+
|atom_index_0|count_label|         avg_label|max_label|min_label| stddev_samp_label|
+------------+-----------+------------------+---------+---------+------------------+
|          28|         72| 21.92878094444444|  9.81731|   -1.058| 35.23185395113022|
|          11|     334284|18.568073696246394|  99.9989|-0.000105| 35.41070960961889|
|          24|       5293| 18.07532229132819|  99.5894|-0.002885|35.300296991256936|
|          22|      22907|17.925526548784262|  99.8763|-0.000384| 36.71707042321272|
|          26|        780|17.898671452564102|  92.6664|-0.001858| 33.71423610380489|
|          20|      70570| 17.23977482749044|  99.9721|-0.000213| 36.33419490119731|
|          23|       7704|16.949363009216015|  99.7316| -0.00081| 34.72306937629732|
|          18|     159787|16.876985383717113|   99.988|-0.000156| 36.11973690200501|
|          19|     107542|16.633819243374695|  99.9633|-0.000131|

In [77]:
aggrList01(1).show(100)

+------------+-----------+--------------------+---------+---------+------------------+
|atom_index_1|count_label|           avg_label|max_label|min_label| stddev_samp_label|
+------------+-----------+--------------------+---------+---------+------------------+
|           0|     242634|   46.90514616430927|   99.993|-0.000164| 42.96695017831453|
|           8|     186355|  23.482594417557937|  99.9792|-0.000113|43.352403360702404|
|           7|     269810|  20.872959390645292|  99.9997|-0.000105| 39.59035888296813|
|           6|     294535|  20.297015414748614|  99.9933|-0.000165| 38.20378761245526|
|           5|     311155|  19.818153592408976|  99.9997| -0.00011|  37.6973580993219|
|           4|     329888|  18.534412229899093|  99.9983|-0.000105| 36.32004309035174|
|           3|     343860|   17.71841680886415|  99.9998|-0.000122|35.593571046460035|
|           2|     403060|  16.376890409715777|    9e-06|-0.000168| 33.09967877556212|
|           1|     387849|    10.5690342317

In [91]:
trainingData.transform(recategorizeAtomIndex1Column).show

+-------+----------------+------------+------------+----+---------+--------------+
|     id|   molecule_name|atom_index_0|atom_index_1|type|    label|atom_index_1_i|
+-------+----------------+------------+------------+----+---------+--------------+
|      0|dsgdb9nsd_000001|           1|           0|1JHC|  84.8076|             0|
|      1|dsgdb9nsd_000001|           1|           2|2JHH|  -11.257|             7|
|     10|dsgdb9nsd_000002|           1|           0|1JHN|  32.6889|             0|
|    100|dsgdb9nsd_000012|           3|           5|3JHH|  12.0536|             4|
|  10000|dsgdb9nsd_000524|          14|           5|1JHC|   86.787|             4|
| 100000|dsgdb9nsd_003735|           7|           8|2JHH| -11.7083|             1|
|1000002|dsgdb9nsd_034940|          15|           5|2JHC|-0.342991|             4|
|1000003|dsgdb9nsd_034940|          15|           6|2JHC| -1.57302|             3|
|1000004|dsgdb9nsd_034940|          15|           7|1JHC|   104.79|             2|
|100

In [73]:
aggrList01(2).show

+---+-----------+-------------------+---------+---------+------------------+
| t1|count_label|          avg_label|max_label|min_label| stddev_samp_label|
+---+-----------+-------------------+---------+---------+------------------+
|  1|     565157|  92.23050224450911|  99.9998|      100|21.054318656302712|
|  3|    1699854|   3.77302948605652|    9e-06|-0.000105| 3.288990500901999|
|  2|    1228946|-2.3371433076855834|  9.99992|-0.000105| 6.210650967789539|
+---+-----------+-------------------+---------+---------+------------------+



In [75]:
aggrList01(3).show

+---+-----------+-------------------+---------+---------+------------------+
| t4|count_label|          avg_label|max_label|min_label| stddev_samp_label|
+---+-----------+-------------------+---------+---------+------------------+
|  C|    2520945| 21.627150141810457|  99.9998|-0.000105|39.054519662869666|
|  N|     246166|  7.918650293602685|    9e-06|-0.000109|16.171814313437576|
|  H|     726846|-1.1062428366352648|    9e-06|-0.000133| 8.277172044205791|
+---+-----------+-------------------+---------+---------+------------------+



+ Define reacategorization function

In [144]:
def recategorizeAtomIndex1Column(df:DataFrame):DataFrame = df.select(col("*"), 
                                                                     when(col("atom_index_1") ==="0", 0).
                                                                     when(col("atom_index_1") ==="8", 1).
                                                                     when(col("atom_index_1") ==="7", 2).
                                                                     when(col("atom_index_1") ==="6", 3).
                                                                     when(col("atom_index_1") ==="5", 4).
                                                                     when(col("atom_index_1") ==="4", 5).
                                                                     when(col("atom_index_1") ==="3", 6).
                                                                     when(col("atom_index_1") ==="2", 7).
                                                                     when(col("atom_index_1") ==="1", 8).
                                                                     when(col("atom_index_1") ==="12", 9).
                                                                     when(col("atom_index_1") ==="13", 10).
                                                                     when(col("atom_index_1") ==="14", 11).
                                                                     when(col("atom_index_1") ==="15", 12).
                                                                     when(col("atom_index_1") ==="16", 13).
                                                                     when(col("atom_index_1") ==="17", 14).
                                                                     when(col("atom_index_1") ==="18", 15).
                                                                     when(col("atom_index_1") ==="19", 16).
                                                                     when(col("atom_index_1") ==="20", 17).
                                                                     when(col("atom_index_1") ==="21", 18).
                                                                     when(col("atom_index_1") ==="22", 19).
                                                                     when(col("atom_index_1") ==="23", 20).
                                                                     when(col("atom_index_1") ==="24", 21).
                                                                     when(col("atom_index_1") ==="25", 22).
                                                                     when(col("atom_index_1") ==="26", 23).
                                                                     when(col("atom_index_1") ==="27", 24).
                                                                     when(col("atom_index_1") ==="28", 25).
                                                                     otherwise(26).as("atom_index_1_i"))
preprocessingList += recategorizeAtomIndex1Column
recategorizeAtomIndex1Column(trainingData).show

+-------+----------------+------------+------------+----+---------+--------------+
|     id|   molecule_name|atom_index_0|atom_index_1|type|    label|atom_index_1_i|
+-------+----------------+------------+------------+----+---------+--------------+
|      0|dsgdb9nsd_000001|           1|           0|1JHC|  84.8076|             0|
|      1|dsgdb9nsd_000001|           1|           2|2JHH|  -11.257|             7|
|     10|dsgdb9nsd_000002|           1|           0|1JHN|  32.6889|             0|
|    100|dsgdb9nsd_000012|           3|           5|3JHH|  12.0536|             4|
|  10000|dsgdb9nsd_000524|          14|           5|1JHC|   86.787|             4|
| 100000|dsgdb9nsd_003735|           7|           8|2JHH| -11.7083|             1|
|1000002|dsgdb9nsd_034940|          15|           5|2JHC|-0.342991|             4|
|1000003|dsgdb9nsd_034940|          15|           6|2JHC| -1.57302|             3|
|1000004|dsgdb9nsd_034940|          15|           7|1JHC|   104.79|             2|
|100

defined function recategorizeAtomIndex1Column
res143_1: ListBuffer[DataFrame => DataFrame] = ListBuffer(
  <function1>,
  <function1>,
  <function1>
)

#### Re-define categorical columns

In [94]:
categoricalCols = categoricalCols.filterNot(_=="atom_index_1")

### Preprocessing before applying pipeline (TODO: search for how to build custome pipeline)

In [153]:
def convertBeforePipeline(df:DataFrame):DataFrame ={
    println("Convert DataFrame by " + preprocessingList.length.toString + " defined processes.")
    preprocessingList.foldLeft(df)((df,process) => df.transform(process))
}
val allConvertedColumns = convertBeforePipeline(trainingData).columns

Convert DataFrame by 3 defined processes.


defined function convertBeforePipeline
allConvertedColumns: Array[String] = Array(
  "id",
  "molecule_name",
  "atom_index_0",
  "atom_index_1",
  "label",
  "t1",
  "t4",
  "atom_index_diff",
  "atom_index_1_i"
)

In [149]:
convertBeforePipeline(trainingData).show

Convert DataFrame by 3 defined processes.
+-------+----------------+------------+------------+---------+---+---+---------------+--------------+
|     id|   molecule_name|atom_index_0|atom_index_1|    label| t1| t4|atom_index_diff|atom_index_1_i|
+-------+----------------+------------+------------+---------+---+---+---------------+--------------+
|      0|dsgdb9nsd_000001|           1|           0|  84.8076|  1|  C|            1.0|             0|
|      1|dsgdb9nsd_000001|           1|           2|  -11.257|  2|  H|            1.0|             7|
|     10|dsgdb9nsd_000002|           1|           0|  32.6889|  1|  N|            1.0|             0|
|    100|dsgdb9nsd_000012|           3|           5|  12.0536|  3|  H|            2.0|             4|
|  10000|dsgdb9nsd_000524|          14|           5|   86.787|  1|  C|            9.0|             4|
| 100000|dsgdb9nsd_003735|           7|           8| -11.7083|  2|  H|            1.0|             1|
|1000002|dsgdb9nsd_034940|          15| 

### Build Pipeline

#### StringIndexer

In [150]:
import org.apache.spark.ml.feature.{OneHotEncoderEstimator, StringIndexer}
val newCategoricalCols = Seq("atom_index_0_c", "t1_c", "t4_c")
val categoricalColsMap = categoricalCols.zip(newCategoricalCols).toMap
val stringIndexerStages = categoricalCols.
    map(c => new StringIndexer().setInputCol(c).setOutputCol(categoricalColsMap(c)).fit(trainingData.transform(x => processTypeColumn(x)).select(c)))

import org.apache.spark.ml.feature.{OneHotEncoderEstimator, StringIndexer}

newCategoricalCols: Seq[String] = List("atom_index_0_c", "t1_c", "t4_c")
categoricalColsMap: Map[String, String] = Map(
  "atom_index_0" -> "atom_index_0_c",
  "t1" -> "t1_c",
  "t4" -> "t4_c"
)
stringIndexerStages: Array[org.apache.spark.ml.feature.StringIndexerModel] = Array(
  strIdx_584660428c97,
  strIdx_d4ea32bae3f8,
  strIdx_a726a61031a0
)

In [150]:
// def transformStringIndex(df:DataFrame, colsMap:Map[String, String]):DataFrame = {
//     colsMap.foldLeft(df)((d, m) => {
//         val si = new StringIndexer().setInputCol(m._1).setOutputCol(m._2)
//         val sm = si.fit(d)
//         sm.transform(d)
//     })
// }

### Modeling

#### import modules

In [173]:
import org.apache.spark.ml.regression.{LinearRegression, LinearRegressionModel}
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.tuning.CrossValidator
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.regression.GeneralizedLinearRegression
val MaxIter: Seq[Int] = Seq(1000)
val RegParam: Seq[Double] = Seq(0.001)
val Tol: Seq[Double] = Seq(1e-6)
val ElasticNetParam: Seq[Double] = Seq(0.001)

import org.apache.spark.ml.regression.{LinearRegression, LinearRegressionModel}

import org.apache.spark.ml.evaluation.RegressionEvaluator

import org.apache.spark.ml.tuning.ParamGridBuilder

import org.apache.spark.ml.tuning.CrossValidator

import org.apache.spark.mllib.evaluation.RegressionMetrics

import org.apache.spark.ml.regression.GeneralizedLinearRegression

MaxIter: Seq[Int] = List(1000)
RegParam: Seq[Double] = List(0.001)
Tol: Seq[Double] = List(1.0E-6)
ElasticNetParam: Seq[Double] = List(0.001)

### Linear Regression (similar way to glmnet in R)

In [161]:
val featureCols = allConvertedColumns.
    filter(_.matches("""^(atom|t).*""")).
    filterNot(_.matches("""atom_index_1""")).
    map(x => categoricalColsMap.getOrElse(x, x))

featureCols: Array[String] = Array(
  "atom_index_0_c",
  "t1_c",
  "t4_c",
  "atom_index_diff",
  "atom_index_1_i"
)

In [162]:
val assembler = new VectorAssembler().setInputCols(featureCols).setOutputCol("features")
val model = new LinearRegression().setFeaturesCol("features").setLabelCol("label")

assembler: VectorAssembler = vecAssembler_1c539e63fcc8
model: LinearRegression = linReg_5e540c6ee1e1

In [163]:
val pipeline = new Pipeline().setStages(stringIndexerStages :+ assembler :+ model)

pipeline: Pipeline = pipeline_2bd720c74de0

In [164]:
val paramGrid = new ParamGridBuilder().
    addGrid(model.maxIter, MaxIter).
    addGrid(model.regParam, RegParam).
    addGrid(model.tol, Tol).
    addGrid(model.elasticNetParam, ElasticNetParam).build()

val cv = new CrossValidator()
    .setEstimator(pipeline)
    .setEvaluator(new RegressionEvaluator)
    .setEstimatorParamMaps(paramGrid)
    .setNumFolds(numFolds)

paramGrid: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	linReg_5e540c6ee1e1-elasticNetParam: 0.001,
	linReg_5e540c6ee1e1-maxIter: 1000,
	linReg_5e540c6ee1e1-regParam: 0.001,
	linReg_5e540c6ee1e1-tol: 1.0E-6
}
)
cv: CrossValidator = cv_46d0fe37d68e

In [169]:
val cvModel = cv.fit(convertBeforePipeline( trainingData))

Convert DataFrame by 3 defined processes.


19/07/15 12:32:31 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
19/07/15 12:32:31 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
19/07/15 12:32:53 INFO OWLQN: Step Size: 0.1283
19/07/15 12:32:53 INFO OWLQN: Val and Grad Norm: 0.422005 (rel: 0.156) 0.669705
19/07/15 12:32:53 INFO OWLQN: Step Size: 0.5000
19/07/15 12:32:53 INFO OWLQN: Val and Grad Norm: 0.281835 (rel: 0.332) 0.338823
19/07/15 12:32:53 INFO OWLQN: Step Size: 0.5000
19/07/15 12:32:53 INFO OWLQN: Val and Grad Norm: 0.241943 (rel: 0.142) 0.162320
19/07/15 12:32:53 INFO OWLQN: Step Size: 0.5000
19/07/15 12:32:53 INFO OWLQN: Val and Grad Norm: 0.231815 (rel: 0.0419) 0.0831343
19/07/15 12:32:53 INFO OWLQN: Step Size: 0.5000
19/07/15 12:32:53 INFO OWLQN: Val and Grad Norm: 0.229180 (rel: 0.0114) 0.0461268
19/07/15 12:32:53 INFO OWLQN: Step Size: 0.5000
19/07/15 12:32:53 INFO OWLQN: Val and Grad Norm: 0.228186 (rel: 0.00434) 0.0271662
19/07/15 12:32

cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_46d0fe37d68e

In [170]:
import org.apache.spark.sql.Row
println("Evaluating model on train and validation set and calculating RMSE")
val trainPredictionsAndLabels = cvModel.
    transform(convertBeforePipeline(trainingData)).
    select("label", "prediction").
    map { case Row(label: Double, prediction: Double) => (label, prediction) }.rdd

Evaluating model on train and validation set and calculating RMSE
Convert DataFrame by 3 defined processes.


import org.apache.spark.sql.Row

trainPredictionsAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1121] at rdd at cmd169.sc:5

In [172]:
val validPredictionsAndLabels = cvModel.transform(convertBeforePipeline(validationData)).
    select("label", "prediction").
    map { case Row(label: Double, prediction:Double) => (label, prediction) }.rdd

val trainRegressionMetrics = new RegressionMetrics(trainPredictionsAndLabels) 
val validRegressionMetrics = new RegressionMetrics(validPredictionsAndLabels)

val bestModel = cvModel.bestModel.asInstanceOf[PipelineModel]

Convert DataFrame by 3 defined processes.


validPredictionsAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1131] at rdd at cmd171.sc:3
trainRegressionMetrics: RegressionMetrics = org.apache.spark.mllib.evaluation.RegressionMetrics@16fc79b1
validRegressionMetrics: RegressionMetrics = org.apache.spark.mllib.evaluation.RegressionMetrics@7c6c0001
bestModel: PipelineModel = pipeline_2bd720c74de0

In [174]:
val results = "n=====================================================================n" + 
    s"Param trainSample: ${trainSample}n" +
    s"Param testSample: ${testSample}n" +
    s"TrainingData count: ${trainingData.count}n" +
    s"ValidationData count: ${validationData.count}n" +
    s"TestData count: ${testData.count}n" +
    "=====================================================================n" +
    s"Param maxIter = ${MaxIter.mkString(",")}n" +
    s"Param numFolds = ${numFolds}n" +
    "=====================================================================n" +
    s"Training data MSE = ${trainRegressionMetrics.meanSquaredError}n" +
    s"Training data RMSE = ${trainRegressionMetrics.rootMeanSquaredError}n" +
    s"Training data R-squared = ${trainRegressionMetrics.r2}n" +
    s"Training data MAE = ${trainRegressionMetrics.meanAbsoluteError}n" +
    s"Training data Explained variance = ${trainRegressionMetrics.explainedVariance}n" +
    "=====================================================================n" +
    s"Validation data MSE = ${validRegressionMetrics.meanSquaredError}n" +
    s"Validation data RMSE = ${validRegressionMetrics.rootMeanSquaredError}n" +
    s"Validation data R-squared = ${validRegressionMetrics.r2}n" +
    s"Validation data MAE = ${validRegressionMetrics.meanAbsoluteError}n"+
    s"Validation data Explained variance = ${validRegressionMetrics.explainedVariance}n" +
    s"CV params explained: ${cvModel.explainParams}n" +
    s"LR params explained: ${bestModel.stages.last.asInstanceOf[LinearRegressionModel].explainParams}n "+ 
    "=====================================================================n"

results: String = """n=====================================================================nParam trainSample: 1.0nParam testSample: 1.0nTrainingData count: 3493957nValidationData count: 1164190nTestData count: 2505542n=====================================================================nParam maxIter = 1000nParam numFolds = 10n=====================================================================nTraining data MSE = 555.4301030466736nTraining data RMSE = 23.567564639705nTraining data R-squared = 0.16605207268456756nTraining data MAE = 19.301909893814088nTraining data Explained variance = 1221.491284635936n=====================================================================nValidation data MSE = 555.4205561990009nValidation data RMSE = 23.567362096743047nValidation data R-squared = 0.1654010432442642nValidation data MAE = 19.293976088991744nValidation data Explained variance = 1219.290368650057nCV params explained: estimator: estimator for selection (current: pipeline_2bd720c74de0)
est

In [176]:
println(results)

n=====================================================================nParam trainSample: 1.0nParam testSample: 1.0nTrainingData count: 3493957nValidationData count: 1164190nTestData count: 2505542n=====================================================================nParam maxIter = 1000nParam numFolds = 10n=====================================================================nTraining data MSE = 555.4301030466736nTraining data RMSE = 23.567564639705nTraining data R-squared = 0.16605207268456756nTraining data MAE = 19.301909893814088nTraining data Explained variance = 1221.491284635936n=====================================================================nValidation data MSE = 555.4205561990009nValidation data RMSE = 23.567362096743047nValidation data R-squared = 0.1654010432442642nValidation data MAE = 19.293976088991744nValidation data Explained variance = 1219.290368650057nCV params explained: estimator: estimator for selection (current: pipeline_2bd720c74de0)
estimatorParamMaps: para

In [177]:
println("Run prediction on the test set")
   cvModel.transform(convertBeforePipeline(testData)).
    select("id", "prediction").
    withColumnRenamed("prediction", "loss").
    coalesce(1). // to get all the predictions in a single csv file
    write.format("com.databricks.spark.csv").
    option("header", "true").
    save("output/result_LR.csv")

Run prediction on the test set
Convert DataFrame by 3 defined processes.


In [197]:
val glr = new GeneralizedLinearRegression().
    setMaxIter(1000).
    setRegParam(0.03).
    setFamily("gaussian").
    setLink( "identity" )

glr: GeneralizedLinearRegression = glm_adcd0939f348

In [ ]:
// val glrModel = glr.fit(regressionData)

// val summary = glrModel.summary
// summary.residuals().show()

// println("Residual Degree Of Freedom: " + summary.residualDegreeOfFreedom)
// println("Residual Degree Of Freedom Null: " + summary.residualDegreeOfFreedomNull)
// println("AIC: " + summary.aic)
// println("Dispersion: " + summary.dispersion)
// println("Null Deviance: " + summary.nullDeviance)
// println("Deviance: " +summary.deviance)
// println("p-values: " + summary.pValues.mkString(","))
// println("t-values: " + summary.tValues.mkString(","))
// println("Coefficient Standard Error: " + summary.coefficientStandardErrors.mkString(","))